### Team 2 initial data wrangling.

In this notebook we will try to obtain the waymo data in a way that it is easily processed and with which we can start our data analysis.

Example code obtained from: 
- https://colab.research.google.com/github/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb#scrollTo=JPc-xBE6VMHi

It was adapted in order to work with local windows machine

You also need to follow:

https://github.com/waymo-research/waymo-open-dataset

in order to get the waymo open dataset package.

In [2]:
import os
import imp
import tensorflow as tf
import math
import numpy as np
import itertools
import matplotlib.pyplot as plt
import cv2
from os import listdir
from os.path import isfile, join

# TODO: Change this to your own setting
os.environ['PYTHONPATH']='/env/python:~/github/waymo-open-dataset'
m=imp.find_module('waymo_open_dataset', ['.'])
imp.load_module('waymo_open_dataset', m[0], m[1], m[2])

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [ ]:
tf.enable_eager_execution()
path = 'data/T1/'
files = [f for f in listdir(path) if isfile(join(path,f))]

for i in range(len(files)):
    file = path+files[i]
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    # Load frames from dataset
    frames = []
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        frames.append(frame)

    print("filename:", file, "Num of frames:", len(frames))

    # ["FRONT", "FRONT_LEFT", "SIDE_LEFT", "FRONT_RIGHT", "SIDE_RIGHT"]
    POS = 0

    # Read img from waymo img class into numpy arrays
    imgs = []
    for frame in frames:
      l = list(enumerate(frame.images))
      _, im = l[POS]
      img = tf.image.decode_jpeg(im.image)
      height, width, _ = img.shape
      size = (width,height)
      imgs.append(img.numpy())
    
    output = 'videos/T1/'+ files[i] + ".avi"
    fps = 12

    # Write a sequence of imgs to a video  
    out = cv2.VideoWriter(output, cv2.VideoWriter_fourcc(*'XVID'), fps, size)
    for img in imgs:
        # Get it in GBR instead of BRG
        gbr = img[...,::-1].copy()
        out.write(gbr)
    out.release()

Instructions for updating:
Colocations handled automatically by placer.
filename: data/T1/segment-10206293520369375008_2796_800_2816_800_with_camera_labels.tfrecord Num of frames: 199
filename: data/T1/segment-10241508783381919015_2889_360_2909_360_with_camera_labels.tfrecord Num of frames: 198
filename: data/T1/segment-10500357041547037089_1474_800_1494_800_with_camera_labels.tfrecord Num of frames: 199
